<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/bird2vec_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
!pwd # just show pwd
!mkdir train-data
!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-265.index" ./train-data/model.ckpt.index
!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-265.data-00000-of-00001" ./train-data/model.ckpt.data-00000-of-00001
!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-265.meta" ./train-data/model.ckpt.meta
!cp "/content/gdrive/My Drive/wgtrain/infer/infer.meta" ./train-data/infer.meta


/content


In [13]:
# Load the model
import tensorflow as tf

tf.reset_default_graph()
saver = tf.train.import_meta_graph('./train-data/infer.meta')
graph = tf.get_default_graph()
sess = tf.InteractiveSession()
saver.restore(sess, 'model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model.ckpt


In [0]:
# Generate and display audio

# CHANGE THESE to change number of examples generated/displayed
ngenerate = 64
ndisplay = 4

import numpy as np
import PIL.Image
from IPython.display import display, Audio
import time as time

# Sample latent vectors
_z = (np.random.rand(ngenerate, 100) * 2.) - 1.

In [0]:
# Generate
z = graph.get_tensor_by_name('z:0')

In [0]:
G_z = graph.get_tensor_by_name('G_z:0')[:, :, 0]

In [0]:
G_z_spec = graph.get_tensor_by_name('G_z:0')

In [20]:
start = time.time()
_G_z, _G_z_spec = sess.run([G_z, G_z_spec], {z: _z})
print('Finished! (Took {} seconds)'.format(time.time() - start))

Finished! (Took 5.374453544616699 seconds)


In [21]:
print("z: Type: {} shape: {}".format(type(z),z.shape))
print("G_z: Type: {} shape: {}".format(type(G_z),G_z.shape))
print("G_z_spec: Type: {} shape: {}".format(type(G_z_spec),G_z_spec.shape))
print("_z: Type: {} shape: {}".format(type(z),_z.shape))
print("_G_z: Type: {} shape: {}".format(type(G_z),_G_z.shape))
print("_G_z_spec: Type: {} shape: {}".format(type(_G_z_spec),_G_z_spec.shape))
print("All 100 elements of first latent vector:\n{}".format(_z[0,:]))
print("All 100 elements of second latent vector:\n{}".format(_z[1,0:len(_z[0])]))

z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 100)
G_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 16384)
G_z_spec: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 16384, 1)
_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (64, 100)
_G_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (64, 16384)
_G_z_spec: Type: <class 'numpy.ndarray'> shape: (64, 16384, 1)
All 100 elements of first latent vector:
[ 0.20703196  0.75911589  0.57229685 -0.44530611  0.05360572  0.30907169
 -0.85906442  0.56607118 -0.073252   -0.99735835 -0.13672073 -0.40847082
  0.4093511   0.60860768 -0.3265133  -0.72248087 -0.91794737 -0.97132284
 -0.38930319  0.83468944  0.02937778  0.55530703  0.08274279  0.61908695
 -0.36514104 -0.58730463 -0.9096128   0.43131282  0.89953571  0.95564579
 -0.38406805 -0.30535679 -0.64848072  0.47464078 -0.70345644 -0.45177308
 -0.63952934  0.94948596 -0.16892767 -0.9626317   0.1731318   0.

In [23]:
for i in range(ndisplay):
  print('-' * 80)
  print('Example {}'.format(i))
  display(Audio(_G_z[i], rate=16000))

--------------------------------------------------------------------------------
Example 0


--------------------------------------------------------------------------------
Example 1


--------------------------------------------------------------------------------
Example 2


--------------------------------------------------------------------------------
Example 3
